<a href="https://colab.research.google.com/github/deepakri201/NLSTNatureSciData/blob/main/UsageNotes/NLSTSybilFMDemo/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLST_Sybil_FM_demo_part1

In this notebook, we use metadata extracted from the DICOM SR files (present in BQ tables), and join with the clinical metadata already in IDC.

Deepa Krishnaswamy

Brigham and Women's Hospital

August 2025

Notes:
- Colab Pro
- Tables that hold the DICOM SR metadata: idc-external-018.sr_nlst_sybil.bbox_measurements
- Insert requirements for running with own project
- Need two datasets - nlst_sybil_fm_demo and nlst_sybil_fm_demo

In [ ]:
# SET THESE OPTIONS

# Create the BQ tables to hold the measurements from the SRs
# If not, load from csv file in github
create_bq_tables = 0

### TO DO ###
# Create this BQ dataset if create_bq_tables = 1
dataset_name = "nlst_sybil_fm_demo"


# Parameterization

In [ ]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment Setup

In [ ]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [ ]:
import os
import sys
import time

import numpy as np
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

import json
from pathlib import Path

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

In [ ]:
from idc_index import IDCClient

idc_client = IDCClient.client()

In [ ]:
# if create_bq_tables - download the BQ queries
# else get the bbox_measurements csv file from github

if create_bq_tables:
  !wget -O /content/measurement_groups.sql https://raw.githubusercontent.com/deepakri201/SR_for_NLST_Sybil/main/sql/measurement_groups.sql
  !wget -O /content/bbox_measurements.sql https://raw.githubusercontent.com/deepakri201/SR_for_NLST_Sybil/main/sql/bbox_measurements.sql
else:
  !wget https://github.com/deepakri201/SR_for_NLST_Sybil/releases/download/v1.0.1/bbox_measurements.csv


--2025-08-28 15:14:35--  https://raw.githubusercontent.com/deepakri201/SR_for_NLST_Sybil/main/sql/measurement_groups.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7156 (7.0K) [text/plain]
Saving to: ‘/content/measurement_groups.sql’

/content/measuremen 100%[===================>]   6.99K  --.-KB/s    in 0.001s  

2025-08-28 15:14:35 (10.2 MB/s) - ‘/content/measurement_groups.sql’ saved [7156/7156]

--2025-08-28 15:14:35--  https://raw.githubusercontent.com/deepakri201/SR_for_NLST_Sybil/main/sql/bbox_measurements.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request se

# Get the bounding box information

## df_sr - get the metadata from the SRs

In [ ]:
# If create the BQ tables, run two queries and save the results
# Requires access to the original DICOM SR files and DICOM store
# Else load the measurements from a github release attachment
# MAKE SURE you have a dataset called "sr_nlst_sybil" already created!

if create_bq_tables:

  # Get queries
  # Measurement groups
  query_measurement_groups_filename = "/content/measurement_groups.sql"
  with open(query_measurement_groups_filename, 'r') as file:
    query_measurement_groups = file.read()
  # Bbox measurements
  query_bbox_measurements_filename = "/content/bbox_measurements.sql"
  with open(query_bbox_measurements_filename, 'r') as file:
    query_bbox_measurements = file.read()

  # Create the table below using the query_measurement_groups
  # `idc-external-018.sr_nlst_sybil.measurement_groups`
  client_bq = bigquery.Client(project=project_name)
  destination_table_id_measurement_groups = ".".join([project_name,
                                                      dataset_name,
                                                      "measurement_groups"])
  job_config = bigquery.QueryJobConfig(destination=destination_table_id_measurement_groups)
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
  query_job = client_bq.query(query_measurement_groups, job_config=job_config)
  query_job.result()
  print(f"Query results saved to table: {destination_table_id_measurement_groups}")

  # Create the table below using the query_bbox_measurements
  # `idc-external-018.sr_nlst_sybil.bbox_measurements`
  destination_table_id_bbox_measurements = ".".join([project_name,
                                                     dataset_name,
                                                     "bbox_measurements"])
  job_config = bigquery.QueryJobConfig(destination=destination_table_id_bbox_measurements)
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
  query_job = client_bq.query(query_bbox_measurements, job_config=job_config)
  query_job.result()
  print(f"Query results saved to table: {destination_table_id_bbox_measurements}")

  # Then query that table
  query = f"""
      SELECT
        *
      FROM
        `{project_name}.{dataset_name}.bbox_measurements`
        """
  df_sr = client_bq.query(query).to_dataframe()

else:

  df_sr = pd.read_csv("/content/bbox_measurements.csv")


In [ ]:
# Rename the column so we know it's the SeriesInstanceUID of the SR

# df_sr = df_sr.rename(columns={'SeriesInstanceUID':"SR_SeriesInstanceUID"})

In [ ]:
# First add columns for the width, height, center_x, and center_y

width_list = []
height_list = []
center_x_list = []
center_y_list = []

for index, row in df_sr.iterrows():
  # Get values
  x0 = row['x0']; y0 = row['y0']
  x1 = row['x1']; y1 = row['y1']
  x2 = row['x2']; y2 = row['y2']
  x3 = row['x3']; y3 = row['y3']
  # calculate the width, height and center, as these are needed for display
  min_x = np.min([x0, x1, x2, x3]) # using roi.GraphicData: min_x = np.min([bbox[0], bbox[2], bbox[4], bbox[6]])
  max_x = np.max([x0, x2, x2, x3]) # using roi.GraphicData: max_x = np.max([bbox[0], bbox[2], bbox[4], bbox[6]])
  min_y = np.min([y0, y1, y2, y3]) # using roi.GraphicData: min_y = np.min([bbox[1], bbox[3], bbox[5], bbox[7]])
  max_y = np.max([y0, y1, y2, y3]) # using roi.GraphicData: max_y = np.max([bbox[1], bbox[3], bbox[5], bbox[7]])
  width = max_x - min_x
  height = max_y - min_y
  center_x = min_x + width/2
  center_y = min_y + height/2
  # append
  width_list.append(width)
  height_list.append(height)
  center_x_list.append(center_x)
  center_y_list.append(center_y)

# Add columns
df_sr['width'] = width_list
df_sr['height'] = height_list
df_sr['center_x'] = center_x_list
df_sr['center_y'] = center_y_list

df_sr.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,ReferencedSeriesInstanceUID,trackingIdentifier,trackingUniqueIdentifier,finding,findingSite,ReferencedSOPInstanceUID,...,x1,y1,x2,y2,x3,y3,width,height,center_x,center_y
0,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.826.0.1.3680043.8.498.21399668015831041161...,1.2.840.113654.2.55.24023112856488152536348979...,1,1.2.826.0.1.3680043.8.498.85827078487063079985...,"{\n ""finding"": {\n ""CodeValue"": ""52988006""...","{\n ""findingSite"": {\n ""CodeValue"": ""39607...",1.2.840.113654.2.55.15782756894992033517041215...,...,221.213928,285.038818,221.213928,324.776123,176.891541,324.776123,44.322388,39.737305,199.052734,304.907471
1,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.826.0.1.3680043.8.498.21399668015831041161...,1.2.840.113654.2.55.24023112856488152536348979...,2,1.2.826.0.1.3680043.8.498.97009181801575708672...,"{\n ""finding"": {\n ""CodeValue"": ""52988006""...","{\n ""findingSite"": {\n ""CodeValue"": ""39607...",1.2.840.113654.2.55.20401816777450596174524730...,...,220.000000,287.000000,220.000000,326.000000,177.000000,326.000000,43.000000,39.000000,198.500000,306.500000
2,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.826.0.1.3680043.8.498.21399668015831041161...,1.2.840.113654.2.55.24023112856488152536348979...,3,1.2.826.0.1.3680043.8.498.91833271272521588958...,"{\n ""finding"": {\n ""CodeValue"": ""52988006""...","{\n ""findingSite"": {\n ""CodeValue"": ""39607...",1.2.840.113654.2.55.32049018332506010968782808...,...,220.000000,289.000000,220.000000,327.000000,178.000000,327.000000,42.000000,38.000000,199.000000,308.000000
3,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.826.0.1.3680043.8.498.21399668015831041161...,1.2.840.113654.2.55.24023112856488152536348979...,4,1.2.826.0.1.3680043.8.498.72903306378535207138...,"{\n ""finding"": {\n ""CodeValue"": ""52988006""...","{\n ""findingSite"": {\n ""CodeValue"": ""39607...",1.2.840.113654.2.55.23447016361362557671014824...,...,220.000000,292.000000,220.000000,329.000000,179.000000,329.000000,41.000000,37.000000,199.500000,310.500000
4,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.826.0.1.3680043.8.498.21399668015831041161...,1.2.840.113654.2.55.24023112856488152536348979...,5,1.2.826.0.1.3680043.8.498.81794670554936408072...,"{\n ""finding"": {\n ""CodeValue"": ""52988006""...","{\n ""findingSite"": {\n ""CodeValue"": ""39607...",1.2.840.113654.2.55.29202164341698057454176258...,...,219.000000,294.000000,219.000000,329.000000,179.000000,329.000000,40.000000,35.000000,199.000000,311.500000


In [ ]:
# We need the following fields in order to convert from pixel coordintes to mm for the bounding box
# And we need the Dimensions, Pixel spacing IPP, especially IPP[2] for the z value

referenced_sop_instance_uid_list = list(df_sr['ReferencedSOPInstanceUID'].values)

client_bq = bigquery.Client(project=project_name)

query = f"""
    SELECT
      PatientID,
      StudyInstanceUID,
      SeriesInstanceUID,
      SOPInstanceUID,
      `Rows` as num_rows,
      `Columns` as num_columns,
      PixelSpacing,
      ImagePositionPatient
    FROM
      `bigquery-public-data.idc_current.dicom_all`
    WHERE
      SOPInstanceUID IN UNNEST(@referenced_sop_instance_uid_list)
    ORDER BY
      PatientID,
      StudyInstanceUID,
      SeriesInstanceUID,
      ImagePositionPatient[SAFE_OFFSET(2)]
      """

job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("referenced_sop_instance_uid_list", "STRING", referenced_sop_instance_uid_list)])
df_idc = client_bq.query(query, job_config=job_config).to_dataframe()

In [ ]:
# Reformat the PixelSpacing and the ImagePositionPatient columns

df_idc['pixel_spacing_x'] = [np.float32(f[0]) for f in df_idc['PixelSpacing'].values]
df_idc['pixel_spacing_y'] = [np.float32(f[1]) for f in df_idc['PixelSpacing'].values]
df_idc['ipp0'] = [np.float32(f[0]) for f in df_idc['ImagePositionPatient'].values]
df_idc['ipp1'] = [np.float32(f[1]) for f in df_idc['ImagePositionPatient'].values]
df_idc['ipp2'] = [np.float32(f[2]) for f in df_idc['ImagePositionPatient'].values]

df_idc = df_idc[['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
                 'num_rows', 'num_columns',
                 'pixel_spacing_x', 'pixel_spacing_y',
                 'ipp0', 'ipp1', 'ipp2']]

In [ ]:
df_idc.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,num_rows,num_columns,pixel_spacing_x,pixel_spacing_y,ipp0,ipp1,ipp2
0,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.840.113654.2.55.24023112856488152536348979...,1.2.840.113654.2.55.29991037322734048580038819...,512,512,0.585938,0.585938,-149.707031,-319.707031,-76.400002
1,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.840.113654.2.55.24023112856488152536348979...,1.2.840.113654.2.55.27443508115501826384206327...,512,512,0.585938,0.585938,-149.707031,-319.707031,-78.400002
2,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.840.113654.2.55.24023112856488152536348979...,1.2.840.113654.2.55.16899951679153198601142574...,512,512,0.585938,0.585938,-149.707031,-319.707031,-80.400002
3,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.840.113654.2.55.24023112856488152536348979...,1.2.840.113654.2.55.17481124987277919843449170...,512,512,0.585938,0.585938,-149.707031,-319.707031,-82.400002
4,100012,1.2.840.113654.2.55.23803494144550801138646327...,1.2.840.113654.2.55.24023112856488152536348979...,1.2.840.113654.2.55.29574962348509387538142601...,512,512,0.585938,0.585938,-149.707031,-319.707031,-84.400002


## df_nlst_metadata - get the associated clinical metadata - for classification

In [ ]:
# Here we save an intermediate table with the PatientID, SeriesInstanceUID and SOPInstanceUID
# This will help us to get the associated NLST clinical metadata

client = bigquery.Client(project=project_name, location='US') # since below can't mix US and us-central1
df_needed = df_idc[['PatientID', 'SeriesInstanceUID', 'SOPInstanceUID']].drop_duplicates()
table_id = ".".join([project_name,
                     dataset_name,
                     "needed_uids"])
job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.job.WriteDisposition.WRITE_TRUNCATE
    )
client.load_table_from_dataframe(df_needed, table_id, job_config=job_config).result()

LoadJob<project=idc-external-018, location=US, id=edc31056-9cba-4634-b594-3b88c0479ca4>

In [ ]:
# Here we get the staging data

query = f"""
WITH dicom_mapped AS (
  SELECT
    PatientID,
    StudyInstanceUID,
    StudyDate,
    SeriesInstanceUID,
    SOPInstanceUID,
    InstanceNumber,
    `Rows`,
    `Columns`,
    CASE StudyDate
      WHEN '1999-01-02' THEN 0
      WHEN '2000-01-02' THEN 1
      WHEN '2001-01-02' THEN 2
      ELSE 3
    END AS StudyDate_mapped,
    COUNT(*) OVER (PARTITION BY SeriesInstanceUID) AS sop_count_per_series
  FROM `bigquery-public-data.idc_current.dicom_all`
  WHERE SeriesInstanceUID IN (
    SELECT DISTINCT SeriesInstanceUID
    FROM {table_id}
  )
)

SELECT
  ctab.dicom_patient_id AS PatientID,
  dicom_mapped.StudyInstanceUID,
  dicom_mapped.StudyDate,
  dicom_mapped.SeriesInstanceUID,
  dicom_mapped.SOPInstanceUID,
  ctab.sct_slice_num,
  ctab.study_yr,
  dicom_mapped.Rows,
  dicom_mapped.Columns,
  dicom_mapped.sop_count_per_series,
  CASE prsn.de_stag
    WHEN '110' THEN 0
    WHEN '120' THEN 1
    WHEN '210' THEN 2
    WHEN '220' THEN 3
    WHEN '310' THEN 4
    WHEN '320' THEN 5
    WHEN '400' THEN 6
    ELSE -1
  END AS de_stag_mapped
FROM
  `bigquery-public-data.idc_current_clinical.nlst_ctab` AS ctab
JOIN
  `bigquery-public-data.idc_current_clinical.nlst_prsn` AS prsn
  ON prsn.dicom_patient_id = ctab.dicom_patient_id
JOIN
  dicom_mapped
  ON dicom_mapped.InstanceNumber = ctab.sct_slice_num
  AND ctab.study_yr = dicom_mapped.StudyDate_mapped
JOIN
  {table_id} AS needed
  ON needed.PatientID = prsn.dicom_patient_id
  AND needed.SeriesInstanceUID = dicom_mapped.SeriesInstanceUID
  AND needed.SOPInstanceUID = dicom_mapped.SOPInstanceUID
WHERE
  # Only keep rows where de_stag_mapped is not -1
  CASE prsn.de_stag
    WHEN '110' THEN 0 # "Stage IA"
    WHEN '120' THEN 1 # "Stage IB"
    WHEN '210' THEN 2 # "Stage IIA"
    WHEN '220' THEN 3 # "Stage IIB"
    WHEN '310' THEN 4 # "Stage IIIA"
    WHEN '320' THEN 5 # "Stage IIIB"
    WHEN '400' THEN 6 # "Stage IV"
    ELSE -1
  END != -1
"""
df_nlst_metadata = client_bq.query(query).to_dataframe()

In [ ]:
df_nlst_metadata.head()

,PatientID,StudyInstanceUID,StudyDate,SeriesInstanceUID,SOPInstanceUID,sct_slice_num,study_yr,Rows,Columns,sop_count_per_series,de_stag_mapped
0,118658,1.2.840.113654.2.55.31109105107554913835216649...,1999-01-02,1.2.840.113654.2.55.15851652787098302094885443...,1.2.840.113654.2.55.28846204449130852169602569...,59,0,512,512,262,0
1,122866,1.2.840.113654.2.55.29680888535835067072178042...,2000-01-02,1.2.840.113654.2.55.80557134350152339289139946...,1.2.840.113654.2.55.85140172863189651811817196...,34,1,512,512,152,0
2,209412,1.3.6.1.4.1.14519.5.2.1.7009.9004.144898730588...,1999-01-02,1.3.6.1.4.1.14519.5.2.1.7009.9004.652542111531...,1.3.6.1.4.1.14519.5.2.1.7009.9004.187668884118...,51,0,512,512,204,0
3,106553,1.2.840.113654.2.55.13440616151556421903161182...,2000-01-02,1.2.840.113654.2.55.14660159465432299498263001...,1.2.840.113654.2.55.15814507606219826428921019...,106,1,512,512,139,1
4,103344,1.2.840.113654.2.55.33972947100952262166538956...,2001-01-02,1.2.840.113654.2.55.16282218742406653768564379...,1.2.840.113654.2.55.20018535901410396791740204...,106,2,512,512,156,0


## Join the tables to hold the SR info and clinical metadata info

In [ ]:
# Then join with df_idc
df_sr_join = df_sr.merge(df_idc,
                         left_on=['ReferencedSOPInstanceUID'],
                         right_on=['SOPInstanceUID'],
                         suffixes=('','_right'))
# Drop the duplicate column from the right dataframe
df_sr_join = df_sr_join.drop(columns=['PatientID_right','SOPInstanceUID_right'])

# Then join with the df_nlst_metadata
df_sr_and_nlst = df_sr_join.merge(df_nlst_metadata,
                                  left_on=['ReferencedSOPInstanceUID'],
                                  right_on=['SOPInstanceUID'],
                                  suffixes=('','_right'))
df_sr_and_nlst = df_sr_and_nlst.drop(columns=['PatientID_right','StudyInstanceUID_right','SeriesInstanceUID_right','num_rows', 'num_columns'])
# Rename columns
# df_sr_and_nlst = df_sr_and_nlst.rename({'trackingIdentifier': 'TrackingIdentifier',
#                                         'trackingUniqueIdentifier':'TrackingUID',
#                                         'finding':'FindingType',
#                                         'findingSite': 'FindingSite',
#                                         'ReferencedSOPInstanceUID':'SOPInstanceUID'}, axis=1)
df_sr_and_nlst = df_sr_and_nlst.rename({'trackingIdentifier': 'TrackingIdentifier',
                                        'trackingUniqueIdentifier':'TrackingUID',
                                        'finding':'FindingType',
                                        'findingSite': 'FindingSite'}, axis=1)
# Reorder the columns
df_sr_and_nlst = df_sr_and_nlst[['PatientID', 'StudyInstanceUID', 'StudyDate', 'study_yr', 'SeriesInstanceUID', 'ReferencedSeriesInstanceUID', 'sop_count_per_series',
                                 'TrackingIdentifier', 'TrackingUID', 'ReferencedSOPInstanceUID',
                                 'FindingType', 'FindingSite',
                                 'pixel_spacing_x', 'pixel_spacing_y',
                                 'width', 'height', 'center_x', 'center_y', 'ipp0', 'ipp1', 'ipp2',
                                 'sct_slice_num', 'de_stag_mapped']]
# Order the values in the columns
df_sr_and_nlst = df_sr_and_nlst.sort_values(by=['PatientID', 'study_yr', 'StudyInstanceUID', 'ReferencedSeriesInstanceUID', 'TrackingIdentifier'])
df_sr_and_nlst.head()



,PatientID,StudyInstanceUID,StudyDate,study_yr,SeriesInstanceUID,ReferencedSeriesInstanceUID,sop_count_per_series,TrackingIdentifier,TrackingUID,ReferencedSOPInstanceUID,...,pixel_spacing_y,width,height,center_x,center_y,ipp0,ipp1,ipp2,sct_slice_num,de_stag_mapped
0,100012,1.2.840.113654.2.55.23803494144550801138646327...,1999-01-02,0,1.2.826.0.1.3680043.8.498.57412541352469041701...,1.2.840.113654.2.55.24023112856488152536348979...,162,4,1.2.826.0.1.3680043.8.498.72903306378535207138...,1.2.840.113654.2.55.23447016361362557671014824...,...,0.585938,41.0,37.0,199.5,310.5,-149.707031,-319.707031,-92.400002,38,0
1,100012,1.2.840.113654.2.55.38321092839390108338558865...,2000-01-02,1,1.2.826.0.1.3680043.8.498.93927770228406043617...,1.2.840.113654.2.55.50761756412482430061802871...,157,6,1.2.826.0.1.3680043.8.498.78647028569744757848...,1.2.840.113654.2.55.23050894131215950278743827...,...,0.546875,42.0,39.0,166.0,328.5,-133.726562,-309.726562,1220.699951,39,0
3,100147,1.2.840.113654.2.55.13303292650860633016545772...,1999-01-02,0,1.2.826.0.1.3680043.8.498.95942559036318788688...,1.2.840.113654.2.55.24785488463405747713776937...,110,3,1.2.826.0.1.3680043.8.498.75295883272779976206...,1.2.840.113654.2.55.17981552983033538571417359...,...,0.660156,39.0,37.0,150.5,313.5,-177.300003,-169.000000,-51.189999,88,0
2,100147,1.2.840.113654.2.55.31958452963320032523273261...,2000-01-02,1,1.2.826.0.1.3680043.8.498.44354746155326561489...,1.2.840.113654.2.55.15708941008648745210499888...,116,5,1.2.826.0.1.3680043.8.498.72816859398879583833...,1.2.840.113654.2.55.19851369674160913596156868...,...,0.644531,37.0,37.0,147.5,332.5,-165.000000,-188.899994,-49.275002,92,0
4,100158,1.2.840.113654.2.55.81185422866512279860334872...,2001-01-02,2,1.2.826.0.1.3680043.8.498.24259245170021325341...,1.2.840.113654.2.55.31060976780967844152296392...,146,4,1.2.826.0.1.3680043.8.498.35974924895759750885...,1.2.840.113654.2.55.18324898484332559595555356...,...,0.683594,27.0,28.0,103.5,272.0,-179.100006,-175.000000,-149.440002,57,0


In [ ]:
df_sr_and_nlst.columns

Index(['PatientID', 'StudyInstanceUID', 'StudyDate', 'study_yr',
       'SeriesInstanceUID', 'ReferencedSeriesInstanceUID',
       'sop_count_per_series', 'TrackingIdentifier', 'TrackingUID',
       'ReferencedSOPInstanceUID', 'FindingType', 'FindingSite',
       'pixel_spacing_x', 'pixel_spacing_y', 'width', 'height', 'center_x',
       'center_y', 'ipp0', 'ipp1', 'ipp2', 'sct_slice_num', 'de_stag_mapped'],
      dtype='object')

In [ ]:
# Let's get the counts of the de_stag_mapped
df_sr_and_nlst_counts = df_sr_and_nlst['de_stag_mapped'].value_counts().sort_index()
df_sr_and_nlst_counts

,count
de_stag_mapped,
0,424
1,115
2,21
3,19
4,64
5,48
6,87


# Delete later - temporarily save out csv file to Google Drive

In [ ]:
df_sr_and_nlst.to_csv("/content/nlst_sybil_fm.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp "/content/nlst_sybil_fm.csv" "/content/gdrive/MyDrive/Colab Notebooks/SR_NLST_Sybil/demo/"